In [3]:
from hmac import new
from itertools import count
from operator import length_hint
import pandas as pd
import re

In [2]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

In [19]:
file = '/content/drive/MyDrive/Colab Notebooks/TEst tfm/DATASETS_CDR/CDR_DevelopmentSet.PubTator.txt'

In [5]:
class Blocke:
    
    def __init__(self,texto,entidades,cids):
        self.texto=texto
        self.entidades=entidades
        self.cids=cids

In [6]:
class RE:
    
    def __init__(self,texto,entidad_1_nombre,entidad_1_id,entidad_1_tipo,entidad_2_nombre,entidad_2_id,entidad_2_tipo,relacion):
        self.texto=texto
        self.entidad_1_nombre=entidad_1_nombre
        self.entidad_1_id=entidad_1_id
        self.entidad_1_tipo=entidad_1_tipo
        self.entidad_2_nombre=entidad_2_nombre
        self.entidad_2_id=entidad_2_id
        self.entidad_2_tipo=entidad_2_tipo
        self.relacion=relacion

In [7]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [8]:
def get_block(file):
    dicttext = []
    lines = []
    with open (file) as fp:
        for line in enumerate(fp.readlines()):            
            if line[1] == "\n": 
                dicttext.append(lines)
                lines=[]
            else:                
                lines.append(line)
    return(dicttext)

In [9]:
def texto_unificado(titulo_parser,cuerpo_parser):
    titulo_split = titulo_parser[1].split("|t|")
    cuerpo_split = cuerpo_parser[1].split("|a|")
    return titulo_split[1] + cuerpo_split[1]

In [10]:
def get_list_Block():
    list_bloques = []    
    for item in get_block(file):           
        list_entidades=[]
        list_relations = []
        
        for entity in item[2:]:
            if "CID" in entity[1]:
                get_cid_relations = entity[1].split("CID")
                delete_space=get_cid_relations[1].strip()
                remplazar_comas=delete_space.replace('\t',',')
                array_entity_cid = remplazar_comas.split(',')
                dict_relations = {
                    'entity1_id':array_entity_cid[0],
                    'entity2_id':array_entity_cid[1]
                }
                list_relations.append(dict_relations)                   
            else:
                split_text =entity[1].replace('\t',',')
                clear_text = split_text.strip().split(',')      
                dict_entity = {
                    'id':clear_text[len(clear_text)-1],
                    'tipo':clear_text[len(clear_text)-2],
                    'nombre':clear_text[3]
                }
                list_entidades.append(dict_entity)
        
        bloque = Blocke(texto_unificado(item[0],item[1]),list_entidades,list_relations)
        list_bloques.append(bloque)
    return list_bloques

In [11]:
def get_list_entity():
    list_enty = []
    tupla_parser = []
    for blocke in get_list_Block():
        for entyt in blocke.entidades:
            if entyt['nombre'] not in tupla_parser:
                tupla_parser.append(entyt['nombre'])
                dict_entrys = {
                    'id':entyt['id'],
                    'tipo':entyt['tipo'],
                    'nombre':entyt['nombre']
                }
                list_enty.append(dict_entrys)
           
    return list_enty

In [12]:
def get_data_entity(nombre,list_object):
    id=""
    tipo = ""
    for it in list_object:
        if it['nombre'] == nombre:
               id=it['id']
               tipo = it['tipo']
    
    return id,tipo     

In [13]:
def insert_data_to_list():
    list_object_RE = []
    dict_entidades = get_list_entity()
    for block in get_list_Block():
        text_split = split_into_sentences(block.texto)    
        for cids in block.cids:
            lis_entity_search =[]
            entidad1=[]
            entidad2=[]
            for entity in block.entidades:            
                if entity['id'] == cids['entity1_id']:
                    if entity['nombre'] not in entidad1:
                        entidad1.append(entity['nombre'])
                if entity['id'] == cids['entity2_id']: 
                    if entity['nombre'] not in entidad2:
                        entidad2.append(entity['nombre'])
            
            for oracion in text_split:
                for entidad_1_search in entidad1:
                    for entidad_2_search in entidad2:
                        if oracion.find(entidad_1_search) != -1 and oracion.find(entidad_2_search) != -1:
                            id_entidad1,tipo_entidad_1 =get_data_entity(entidad_1_search,dict_entidades)
                            id_entidad2,tipo_entidad_2 =get_data_entity(entidad_2_search,dict_entidades)
                            newEntityObject = RE(oracion,entidad_1_search,id_entidad1,tipo_entidad_1,entidad_2_search,id_entidad2,tipo_entidad_2,"1")
                            list_object_RE.append(newEntityObject)                            
                        else:
                            id_entidad1,tipo_entidad_1 =get_data_entity(entidad_1_search,dict_entidades)
                            id_entidad2,tipo_entidad_2 =get_data_entity(entidad_2_search,dict_entidades)
                            newEntityObject = RE(oracion,entidad_1_search,id_entidad1,tipo_entidad_1,entidad_2_search,id_entidad2,tipo_entidad_2,"0")
                            list_object_RE.append(newEntityObject)
    return list_object_RE

In [21]:
def main():  
    count=1
    df = pd.DataFrame(columns=['Sentence','entity_1_name','entity_1_id','entity_1_type','entity_2_name','entity_2_id','entity_2_type','label'])
    print('Init ..............')     
    for re_object in insert_data_to_list():
        print('----------------   SENTENCE '+str(count)+'  -----------------')
        print("Sentence: "+re_object.texto)
        print("Entity 1: "+re_object.entidad_1_nombre)
        print("Entity-ID 1: "+re_object.entidad_1_id)
        print("Entity-TIPO 1: "+re_object.entidad_1_tipo)
        print("Entity 2: "+re_object.entidad_2_nombre)
        print("Entity-ID 2: "+re_object.entidad_2_id)
        print("Entity-TIPO 2: "+re_object.entidad_2_tipo)
        print("label: "+re_object.relacion)
        count=count+1
        df.loc[len(df)]=[re_object.texto,re_object.entidad_1_nombre,re_object.entidad_1_id,re_object.entidad_1_tipo,re_object.entidad_2_nombre,re_object.entidad_2_id,re_object.entidad_2_tipo,re_object.relacion]
    df.to_csv('datset.csv',index=False,encoding='utf-8')

In [20]:
main()

Se han truncado las últimas 5000 líneas del flujo de salida.
Entity-TIPO 1: Chemical
Entity 2: elevated intraocular pressure
Entity-ID 2: D009798
Entity-TIPO 2: Disease
label: 0
----------------   ORACION 28140  -----------------
Sentence: 6% of implanted eyes.
Entity 1: fluocinolone acetonide
Entity-ID 1: D005446
Entity-TIPO 1: Chemical
Entity 2: elevated intraocular pressure
Entity-ID 2: D009798
Entity-TIPO 2: Disease
label: 0
----------------   ORACION 28141  -----------------
Sentence: 6% of implanted eyes.
Entity 1: FA
Entity-ID 1: D005446
Entity-TIPO 1: Chemical
Entity 2: elevated intraocular pressure
Entity-ID 2: D009798
Entity-TIPO 2: Disease
label: 0
----------------   ORACION 28142  -----------------
Sentence: Intraocular pressure-lowering surgeries were considered a success (postoperative IOP of 6-21 mm Hg with or without additional IOP-lowering medication) in 85.
Entity 1: fluocinolone acetonide
Entity-ID 1: D005446
Entity-TIPO 1: Chemical
Entity 2: elevated intraocular pre